# Distributed training with Vertex Reduction server

In [1]:
import os
import pprint
import sys
import time

from google.cloud import aiplatform

## Set environment constants

In [2]:
PROJECT_ID = 'jk-mlops-dev'
REGION = 'us-central1'
STAGING_BUCKET = 'gs://jk-vertex-staging'

## Prepare and test a training container

### Create a Dockerfile

In [3]:
#BASE_IMAGE = 'tensorflow/tensorflow:2.5.0-gpu'
BASE_IMAGE = 'gcr.io/deeplearning-platform-release/tf2-gpu.2-5'
CUDA_VERSION='cuda-11-2'
#BASE_IMAGE = 'gcr.io/deeplearning-platform-release/base-cu110'
MODEL_GARDEN_VERSION = '2.5.0'
TRAIN_IMAGE = f'gcr.io/{PROJECT_ID}/model_garden'
TF_TEXT='2.5.0'


dockerfile = f'''
FROM {BASE_IMAGE}

RUN pip install tf-models-official=={MODEL_GARDEN_VERSION} tensorflow-text=={TF_TEXT}

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

ENTRYPOINT ["python"]
CMD ["-c", "print('Hello')"]
'''

with open('Dockerfile', 'w') as f:
    f.write(dockerfile)

### Build a container image

In [4]:
! docker build -t {TRAIN_IMAGE} .

Sending build context to Docker daemon  441.9kB
Step 1/6 : FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-5
latest: Pulling from deeplearning-platform-release/tf2-gpu.2-5

a5e7af40: Already exists 
39a868b3: Already exists 
b10cca85: Already exists 
dd082e0f: Already exists 
5f69b1d6: Already exists 
99ae46a1: Already exists 
2f1f3a5c: Already exists 
f0b7bbf8: Already exists 
f1cb3b81: Already exists 
b1f16acd: Already exists 
4ce81d85: Already exists 
550a4fa4: Already exists 
25ceca25: Already exists 
c506e44a: Already exists 
1d4b2a10: Already exists 
db019315: Already exists 
92ad872e: Already exists 
2ed3db5f: Already exists 
02b5a3a7: Already exists 
60f00749: Already exists 
4706459e: Already exists 
a8f946a1: Already exists 
842bd8ff: Already exists 
988c5196: Already exists 
3d03cb12: Already exists 
345c1107: Already exists 
047dd86b: Already exists 
c3c78abb: Already exists 
5b2f50fb: Already exists 
0539490e: Already exists 
7d8065a0: Already exists 
Digest: sha256:d0

### Push the container to Container Registry

In [5]:
! docker push {TRAIN_IMAGE}

Using default tag: latest
The push refers to repository [gcr.io/jk-mlops-dev/model_garden]

bd7717d0: Preparing 
29ea19d2: Preparing 
464d3f17: Preparing 
daea14d2: Preparing 
b28de254: Preparing 
52e30556: Preparing 
fc085027: Preparing 
7d90a58d: Preparing 
285b3362: Preparing 
0730cb59: Preparing 
18de1f93: Preparing 
d1dfb5d0: Preparing 
686f5924: Preparing 
5de2196f: Preparing 
383a0e80: Preparing 
eaf882b2: Preparing 
2519572d: Preparing 
fbfba824: Preparing 
a8806df6: Preparing 
2a1c8291: Preparing 
b49af22b: Preparing 
b363f69f: Preparing 
0a9a6a11: Preparing 
7e8b38e6: Preparing 
8f196cf4: Preparing 
2e30556: Waiting g 
c085027: Waiting g 
a966f459: Preparing 
b9e63cdf: Preparing 
49f5bf51: Preparing 
730cb59: Waiting g 
325cc380: Preparing 
dd81f9fa: Preparing 
9ea19d2: Pushed   210.9MB/208.2MBv/taxi_classifier_trainer Klatest: digest: sha256:e6afd5a56739a8e96cd865b979a46d3fde905b77325e882a3a9fe3a9b1eb6be9 size: 7461


## Submit Vertext Training jobs

### Define helper functions

In [6]:
def prepare_worker_pool_specs(
    image_uri,
    args,
    cmd, 
    replica_count=1,
    machine_type='n1-standard-4',
    accelerator_count=0,
    accelerator_type='ACCELERATOR_TYPE_UNSPECIFIED'):

    if accelerator_count > 0:
        machine_spec = {
            'machine_type': machine_type,
            'accelerator_type': accelerator_type,
            'accelerator_count': accelerator_count,
        }
    else:
        machine_spec = {
            'machine_type': machine_type
        }
    
    container_spec = {
        'image_uri': image_uri,
        'args': args,
        'command': cmd,
    }
    
    chief_spec = {
        'replica_count': 1,
        'machine_spec': machine_spec,
        'container_spec': container_spec
    }

    worker_pool_specs = [chief_spec]
    if replica_count > 1:
        workers_spec = {
            'replica_count': replica_count - 1,
            'machine_spec': machine_spec,
            'container_spec': container_spec
        }
        worker_pool_specs.append(workers_spec)
    
    return worker_pool_specs

### Prepare worker pool specification

In [7]:
MNLI_TRAIN_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_train.tf_record'
MNLI_VALID_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_valid.tf_record'
BERT_HUB_URL = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'

job_name = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
output_dir = f'gs://jk-vertex-demos/jobs'
model_dir = f'{output_dir}/{job_name}/model'
tfhub_cache_dir = f'{output_dir}/tfhub-cache'
config_file = 'trainer/glue_mnli_matched.yaml'
mode = 'train_and_eval'
experiment = 'bert/sentence_prediction'

machine_type = 'n1-standard-8'
accelerator_count = 1
accelerator_type = 'NVIDIA_TESLA_T4'
strategy = 'mirrored'
#strategy = 'multi_worker_mirrored'
train_steps = 1200
steps_per_loop = 100
summary_interval = 100
validation_interval = 400
checkpoint_interval = 200

replica_count = 1
global_batch_size = 32
all_reduce_alg = 'nccl'

params_override = [
    'task.train_data.input_path=' + MNLI_TRAIN_SPLIT,
    'task.validation_data.input_path=' + MNLI_VALID_SPLIT,
    'task.train_data.global_batch_size=' + str(global_batch_size),
    'task.validation_data.global_batch_size=' + str(global_batch_size),
    'task.hub_module_url=' + BERT_HUB_URL,
    'runtime.num_gpus=' + str(accelerator_count),
    'runtime.distribution_strategy=' + strategy,
    'runtime.all_reduce_alg=' + all_reduce_alg,
    'trainer.train_steps=' + str(train_steps),
    'trainer.steps_per_loop=' + str(steps_per_loop),
    'trainer.summary_interval=' + str(summary_interval),
    'trainer.validation_interval=' + str(validation_interval),
    'trainer.checkpoint_interval=' + str(checkpoint_interval),
]


cmd = [
    "python", "trainer/train.py"
]
args = [
    '--experiment=' + experiment,
    '--mode=' + mode,
    '--model_dir=' + model_dir,
    '--config_file=' + config_file,
    '--tfhub_cache_dir=' + tfhub_cache_dir,
    '--params_override=' + ','.join(params_override),
]

worker_pool_specs = prepare_worker_pool_specs(
    image_uri=TRAIN_IMAGE,
    args=args,
    cmd=cmd,
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_count=accelerator_count,
    accelerator_type=accelerator_type
)

pp = pprint.PrettyPrinter()
print(pp.pformat(worker_pool_specs))

[{'container_spec': {'args': ['--experiment=bert/sentence_prediction',
                              '--mode=train_and_eval',
                              '--model_dir=gs://jk-vertex-demos/jobs/JOB_20210610_143539/model',
                              '--config_file=trainer/glue_mnli_matched.yaml',
                              '--tfhub_cache_dir=gs://jk-vertex-demos/jobs/tfhub-cache',
                              '--params_override=task.train_data.input_path=gs://jk-vertex-demos/datasets/MNLI/mnli_train.tf_record,task.validation_data.input_path=gs://jk-vertex-demos/datasets/MNLI/mnli_valid.tf_record,task.train_data.global_batch_size=32,task.validation_data.global_batch_size=32,task.hub_module_url=https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4,runtime.num_gpus=1,runtime.distribution_strategy=mirrored,runtime.distribution_strategy=multi_worker_mirrored,runtime.all_reduce_alg=nccl,trainer.train_steps=1200,trainer.steps_per_loop=100,trainer.summary_interval=100,trainer.

### Submit and monitor the job

In [8]:
PROJECT_ID = 'jk-mlops-dev'
REGION = 'us-west1'
STAGING_BUCKET = 'gs://jk-vertex-staging'

aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

In [25]:
display_name = job_name

job = aiplatform.CustomJob(
    display_name=display_name,
    worker_pool_specs=worker_pool_specs,
)

job.run(sync=False)

INFO:google.cloud.aiplatform.jobs:Creating CustomJob
INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/895222332033/locations/us-west1/customJobs/3967706456085495808
INFO:google.cloud.aiplatform.jobs:To use this CustomJob in another session:
INFO:google.cloud.aiplatform.jobs:custom_job = aiplatform.CustomJob.get('projects/895222332033/locations/us-west1/customJobs/3967706456085495808')
INFO:google.cloud.aiplatform.jobs:View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-west1/training/3967706456085495808?project=895222332033
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/3967706456085495808 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/3967706456085495808 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-west1/customJobs/3519598

In [ ]:
job.resource_name

In [ ]:
job.wait()

### Test the container image locally

In [ ]:
MNLI_TRAIN_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_train.tf_record'
MNLI_VALID_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_valid.tf_record'
BERT_HUB_URL = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'

num_gpus = 2 
strategy = 'mirrored'
#strategy = 'multi_worker_mirrored'

params_override = [
    'task.train_data.input_path=' + MNLI_TRAIN_SPLIT,
    'task.validation_data.input_path=' + MNLI_VALID_SPLIT,
    'task.hub_module_url=' + BERT_HUB_URL,
    'runtime.num_gpus=' + str(num_gpus),
    'runtime.distribution_strategy=' + strategy,
]

params = ','.join(params_override)

In [ ]:
!docker run -it --rm --gpus all {TRAIN_IMAGE} trainer/train.py \
--experiment=bert/sentence_prediction \
--mode=train_and_eval \
--model_dir={STAGING_BUCKET}/test \
--config_file=trainer/glue_mnli_matched.yaml \
--params_override={params}  
